# Basic Testing of FabSim3 using a Jupyter notebook

This short tutorial shows you how you can use FabSim3 from inside a Jupyter notebook, and demonstrates how you can run simple jobs both locally, and using the ARCHER2 supercomputer.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

## Setting up FabSim3

We will assume you have FabSim3 installed. If not, you can find instructions [here](https://fabsim3.readthedocs.io/en/latest/). 

To test whether FabSim3 works and try out some simple commands, you will need to install the FabDummy plugin. Simply run 

```
fabsim localhost install_plugin:FabDummy
```

This will install the plugin in `FabSim3/plugins/FabDummy`.

You will now need to configure the FabSim3 script, config and `machines_user.yml` file.

**machines_user.yml**

This file is located in `FabSim3/fabsim/deploy`. Open it. This is an important file that lets you configure your run environment for various machines, both your `localhost` as well as remote machines. My `localhost` machine is posted below. In yours, also add the last line that says `ade_exec_school`.

```python
localhost:
  username: wouter
  partition_name: "standard"
  qos_name: "short"
  runtime: 10
  nodes: 1
  ranks: 1
  cores_per_rank: 1
  home_path_template: "/home/wouter/FabSim3/localhost_exe"
```

We will set the following FabSim3 flags:

In [2]:
# FabSim3 arguments
ARGUMENTS = 'dummy_test,cores=1'
# machine to run ensemble on
MACHINE = "localhost"

### FabSim3-Python interface

FabSim3 is a command line tool, but it features a Python interface that is simply a shell over the commandline tools. It is stored locally here. To import it use

In [3]:
# Import the FabSim3 commandline interface
import fabsim3_cmd_api as fab

Using FabSim3 through the API is very similar to using it over the command line. However, one important difference is that standard output (stdout) will _not_ be shown in a notebook environment, but standard error (stderr) output will be shown. In addition, the API contains a few extra shorthand functions to make life easier for you.

In general, we are now ready to perform a test, in this case to `localhost`. The command `fab.fabsim()` can be used to run any FabSim3 command. It takes three arguments:

* command: the name of the Fabsim3 command you seek to use.
* arguments: a string containing the arguments of the command as you would type them on the command line.
* machine: a string containing the label of the machine you wish to use, e.g. 'localhost' or 'archer2'.

**Note**: if you get some type of ssh error, you might have to ssh into the (remote) machine once via the command line before executing `fab.fabsim`.

To run a single dummy job, using the `dummy` command, you simply write the following:

In [4]:
fab.fabsim('dummy', ARGUMENTS, MACHINE)

Executing fabsim localhost dummy:dummy_test,cores=1


This will check every minute on the status of the jobs on the remote host, and sleep otherwise, halting further execution of the script. On the localhost this command doesn't do anything.

In [5]:
# wait for job to complete
fab.wait(machine=MACHINE)

True

To get the results without verifying the presence of output files, call `fab.fetch_results(machine=MACHINE)`. You can also call `fab.clear_results(machine, name_results_dir)` to remove a specific FabSim results directory on a given machine.

In [6]:
fab.fetch_results("localhost")

# clear results in Fabsim directory, change path to your FabSim3 directory
# fab.clear_results("localhost", '/home/wouter/FabSim3/results/')  # localhost
# fab.clear_results("archer2", '/work/e723/e723/edeling/FabSim/')  # archer2

Executing fabsim localhost fetch_results


True

## Visualising output

The built-in fetch_results function above retrieves the output of the job you've submitted, and it will have added a directory named `dummy_test_localhost_1` in your FabSim3 results folder.

To print the contents of the out.txt file within that directory, we've added a simple print function, which you can run as follows:

In [8]:
fab.fabsim('print_dummy_output','dummy_test_localhost_1','localhost')

Executing fabsim localhost print_dummy_output:dummy_test_localhost_1


File Content:
 This dummy test has been concluded with great success.
Tue  8 Oct 09:42:37 BST 2024



## Running an ensemble of dummies

Hopefully you have been able to run the job above, and print its output, without too much difficulty. In this next part we will explain how to run ensembles using FabSim3.

If you look at the input files for the `dummy_test` configuration (viewable online here: https://github.com/djgroen/FabDummy/tree/master/config_files/dummy_test) then you will find a `dummy.txt` file and a directory named `SWEEP`. 
* When you were running the dummy job above, the `dummy.txt` file was used as input for the dummy printer.
* The `SWEEP/` directory however, contains input for every instance if you decide to run the configuration as an ensemble.
  * Each subdirectory in `SWEEP/` contains input files for one instance in the ensemble, while files in the main config directory (in this case `dummy.txt`) will be passed on as input for _all_ instances in the ensemble.
 
To start an ensemble locally using these inputs, and without replicas, you can use the following command:

In [ ]:
fab.fabsim('dummy_ensemble','dummy_test,cores=1','localhost')

**Exercises**
* Can you add the commands below, which allow you to wait for completion, and then to fetch the results?
* When you fetch the results, have a look at your results directory too. Do you notice anything unusual about the structure of your run outputs?

**Exercise**
Now that the job is completed and the results are fetched, you will want to use the `print_dummy_output` mentioned above once more to print the output of specific dummy_test instances. However, you may notice that the structure of your results directory is a bit different now. 
* Do you know what results path you need to use to print the output of the `d2` instance? (hint: your argument will be containing at least one '/' to denote a subpath if you get it to work correctly.)

## Running an ensemble with replicas

Now that you are able to run an ensemble, let's replicate all the instances 5 times! Fortunately this can be done quite easily, by adding the `replicas=5` argument to your dummy_ensemble argument. See the modified function below.

(_Note:_ the `replicas=5` argument trick works for any ensemble execution function in FabSim3, not just `dummy_ensemble`.) 

In [ ]:
fab.fabsim('dummy_ensemble','dummy_test,cores=1,replicas=5','localhost')

**Last little exercise**
As above, add a function to wait for completion and one to fetch the results again. Look at the directory structure of your ensemble run results, and notice how different subdirectories with outputs have been created for each replica.
* Do you know what results path you need to use to print the output of the 4th replica of the `d3` instance of the ensemble?

## Assigment: repeat on Archer2

If you have aquired access to Archer2, you can redo this exercise, except run the ensemble on Archer2 instead on your laptop. To do so you you will need to:

* Copy the advection-diffusion model to a location in Archer2. From within this path, you can do so via `scp -r advection_diffusion_model/ <username>@login.archer2.ac.uk:~/`, replacing `<username>` with your archer2 user name. This will copy `advection_diffusion_model/*` to your Archer2 home directory.
* Configure Archer2 in your `machines_user.yml` file. My file is posted below. Copy it, replace the archer2 entry in your `machine_user.yml`, and modify where needed, replacing my username with yours, and my project (`e723`) with yours (`ta171`). Note that for larger models, this is also where you can request more than 1 compute node.

```python
archer2:
  username: "edeling"
  project: "e723"
  budget: "e723-edeling"
  manual_ssh: true
  remote: "archer2"
  job_wall_time: '0-00:10:00'
  partition_name: "standard"
  ade_exec_school : "/home/e723/e723/edeling/advection_diffusion_model/advection_diffusion.py"
  qos_name: "short"
  runtime: 10
  nodes: 1
  ranks: 1
  cores_per_rank: 1
  run_prefix_commands: ["export PYTHONUSERBASE=/work/$project/$project/$username/.local", "export PATH=$PYTHONUSERBASE/bin:$PATH", "export PYTHONPATH=$PYTHONUSERBASE/lib/python3.9/site-packages:$PYTHONPATH"]
  modules:
        loaded: ["cray-python/3.9.13.1"]
        unloaded: ["cray-python"]
```

* Once Archer2 if configured properly, switching from local ensemble execution to execution on Archer2 is done by simply changing the `MACHINE` flag in this notebook from `"localhost"` to `"archer2"`.